# INF-616 - Exercício 2 - Aula 3/4: *support vector machines*

Professor: Ricardo da Silva Torres -- rtorres@ic.unicamp.br

Professor: Alexandre Ferreira -- melloferreira@ic.unicamp.br  

Monitor: Lucas David -- lucasolivdavid@gmail.com

Este *notebook* faz parte da disciplina INF-616 no curso de extensão MDC.  
Demais artefatos podem ser encontrados no moodle da disciplina: 
[moodle.lab.ic.unicamp.br/332](https://moodle.lab.ic.unicamp.br/moodle/course/view.php?id=332)

Instituto de Computação - Unicamp 2019

In [ ]:
from __future__ import print_function

from math import ceil

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.model_selection import train_test_split

import seaborn as sns

from IPython.display import display

In [ ]:
np.random.seed(1082141)
sns.set()

## Classificando imagens de dígitos
### Lendo o conjunto de dados

**Pen-Based Recognition of Handwritten Digits Data Set**
é um banco de imagens simples e bem conhecido em reconhecimento de imagens.  
Ele é composto por imagens em escala cinza de 8 por 8 pixels divididas em 10 classes de dígitos.

Uma descrição completa pode ser encontrada no seguinte link: [archive.ics.uci.edu/ml/datasets/Pen-Based+Recognition+of+Handwritten+Digits](http://archive.ics.uci.edu/ml/datasets/Pen-Based+Recognition+of+Handwritten+Digits)

In [ ]:
x, y = datasets.load_digits(return_X_y=True)

x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=.5)
print('samples in train: %i' % x_train.shape[0],
      'samples in test: %i' % x_test.shape[0],
      'features: %i' % x_train.shape[1],
      'classes: %i' % (np.max(y_train) + 1),
      sep='\n', end='\n\n')
print(x_train.shape, x_test.shape)

### 64 primeiras amostras no conjunto de treinamento

In [ ]:
plt.figure(figsize=(16, 8))

for ix in range(8  * 32):
    plt.subplot(8, 32, ix + 1)
    plt.imshow(x_train[ix].reshape(8, 8), cmap='Greys')
    plt.axis('off')

### Visualizando o conjunto e frequências das classes

In [ ]:
from sklearn.manifold import TSNE

encoder2D = TSNE()
w_train = encoder2D.fit_transform(x_train)
w_test = encoder2D.fit_transform(x_test)

plt.figure(figsize=(16, 6))
categorical_colors = sns.color_palette()

for ix, (x, y) in enumerate(((w_train, y_train), (w_test, y_test))):
    plt.subplot(1, 2, ix + 1)
    sns.scatterplot(*x.T, hue=y, palette=categorical_colors);

In [ ]:
plt.figure(figsize=(16, 4))

plt.subplot(121)
plt.title('Frequencia das classes no conjunto de treinamento (%i amostras)' % len(x_train))
labels, counts = np.unique(y_train, return_counts=True)
sns.barplot(labels, counts)

plt.subplot(122)
plt.title('Frequencia das classes no conjunto de teste (%i amostras)' % len(x_test))
labels, counts = np.unique(y_test, return_counts=True)
sns.barplot(labels, counts);

### Modelando um classificador de digitos

**Atividade (3 pts):** defina e treine uma máquina de vetor de suporte com kernel RBF, utilizando o scikit-learn.  
Lembre-se que este estimador é extremamente sensível à dados desnormalizados,
o que torna o pre-processamento um passo indispensável.

In [ ]:
# ...

### Avaliando o modelo treinado

In [ ]:
from sklearn import metrics

predictions = sv.predict(x_test)
probabilities = sv.decision_function(x_test)

print(metrics.classification_report(y_test, predictions))

Em problemas envolvendo muitas classes, simplesmente exibir a matriz de confusão
com a função `print` gera uma representação difícil de ler.  
Veja este exemplo:

In [ ]:
metrics.confusion_matrix(y_test, predictions)

Nós podemos melhorar este efeito utilizando um `heatmap`,
onde a grandeza dos valores se torna diretamente proporcional à intensidade da cor adjacente.

**Atividade (1 pt):** calcule a matriz de confusão relativa $R$, que guarda porcentagens de incidências em vez das contagens absolutas. Finalmente, utilize o `heatmap` do seaborn para exibir a matriz alcançada.


Dica: seja $C = \{c_{ij}\}_{10\times 10}$ a matrix de confusão original, $R = \{r_{ij} | r_{ij} := \frac{c_{ij}}{\sum_k c_{ik} }\}$.

In [ ]:
c = metrics.confusion_matrix(y_test, predictions)
# r = ...

plt.figure(figsize=(10, 8))
ax = sns.heatmap(r, linewidths=.5, cmap='YlGnBu', annot=True, fmt='.1%');

**Pergunta (1 pt):** quais dígitos são confundidos com maior frequência no conjunto de teste?

R:

## Support Vector Machine Regressors

O conjunto *Doctor feeds prediction* contém uma relação entre um conjunto de características associadas à um médico atendente e o preço da consulta cobrada. O objetivo é **regredir** este valor o mais próximo possível do valor esperado.   
Ele pode ser encontrado no seguinte link: [kaggle.com/nitin194/doctor-fees-prediction](https://www.kaggle.com/nitin194/doctor-fees-prediction)

In [ ]:
train, test = (pd.read_csv(f'../datasets/doctor-fees/{stage}.csv')
               for stage in ('train', 'test'))

#### Pre-processamento dos dados para um formato mais limpo

- Remove uma linha inválida, contendo `"years experience"` como valor para a coluna qualificação
- Preenche todos os `Place` e `Profile` com valor igual à `NaN` com a tag `unknown`

In [ ]:
def preprocess(frame):
    frame['Rating'] = frame['Rating'].str.replace('%', '').astype(float) / 100.0
    frame['Experience'] = frame['Experience'].str.replace('years experience', '').astype(float)
    frame['Qualification'] = frame['Qualification'].str.replace('[^a-zA-Z]', ' ').str.lower()
    frame['Place'] = frame['Place'].str.replace('[^a-zA-Z]', ' ').str.lower()

preprocess(train)
preprocess(test)

In [ ]:
invalid_row = train['Qualification'].str.contains('years experience')
train = train[~invalid_row]

In [ ]:
train.fillna({'Place': 'unknown', 'Profile': 'unknown'}, inplace=True);
test.fillna({'Place': 'unknown', 'Profile': 'unknown'}, inplace=True);

In [ ]:
train.head()

### Exibindo frequência com que as qualificações, locais e perfis ocorrem nos conjuntos

In [ ]:
def plot_feature_freq(frame, feature, showing=30):
    labels, counts = np.unique(frame[feature].dropna(), return_counts=True)

    # ordena pelas mais frequentes
    p = np.argsort(counts)[::-1]
    labels, counts = labels[p], counts[p]

    g = sns.barplot(labels[:showing], counts[:showing])
    g.set_xticklabels(labels[:showing], rotation=90)
    
    return g

In [ ]:
plt.figure(figsize=(16, 4))

plt.subplot(121)
plot_feature_freq(train, 'Qualification')

plt.subplot(122)
plot_feature_freq(test, 'Qualification')

qualifications, counts = np.unique(train['Qualification'].dropna(), return_counts=True)
p = np.argsort(counts)[::-1]
qualifications = qualifications[p];

In [ ]:
plt.figure(figsize=(16, 4))

plt.subplot(121)
plot_feature_freq(train, 'Place')

plt.subplot(122)
plot_feature_freq(test, 'Place')

places, counts = np.unique(train['Place'].dropna(), return_counts=True)
p = np.argsort(counts)[::-1]
places = places[p];

In [ ]:
plt.figure(figsize=(16, 4))

plt.subplot(121)
plot_feature_freq(train, 'Profile')

plt.subplot(122)
plot_feature_freq(test, 'Profile');

### Modelando um regressor de custo de consulta

Vamos codificar as características categóricas usando o one-hot encoding.  
Entretanto, dado o alto número de ocorrências únicas, nós consideramos somente os 200 valores de maior frequência.

As características contínuas são simplesmente normalizadas com o `StandardScaler`.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

retained_qualif = qualifications[:200].tolist()
retained_places = places[:200].tolist()

qualif_places_enc = OneHotEncoder(categories=[retained_qualif, retained_places], handle_unknown='ignore')
profile_enc = OneHotEncoder()
continuous_enc = make_pipeline(SimpleImputer(strategy='median'),
                               StandardScaler())

encoder = ColumnTransformer([
  ('q_pla', qualif_places_enc, ['Qualification', 'Place']),
  ('prof', profile_enc, ['Profile']),
  ('ex_ra', continuous_enc, ['Experience', 'Rating'])
])

train_e = encoder.fit_transform(train)
test_e = encoder.transform(test)

fee_enc = StandardScaler()
ye_train = fee_enc.fit_transform(train[['Fees']].astype(float)).ravel()

**Atividade (4 pts):** treine dois ou mais regressores --- onde ao menos um é baseado em *máquina de vetor de suporte* --- e reporte o seus respectivos erros quadráticos médios (MSE) sobre as porções de validação separadas. Respeite as seguintes regras:

- Utilize a estratégia 5-3 para fazer a validação cruzada dos resultados e buscar hiperparâmetros
- Busque ao menos dois parâmetros em cada regressor

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
# from sklearn... import ...Regressor

# model = ...
# params = ...
# grid = GridSearchCV(model, params)
# results = cross_validate(...)
#
# model_2 = ...
# params = ...
# grid_2 = GridSearchCV(model, params)
# results_2 = cross_validate(...)
# ...

**Pergunta (1pt):** que estimador apresentou os melhores resultados?

R: